## Steps that weren't done with snakemake 

In [ ]:
## Step 5 : nr blast all contigs with an RdRp hit

# pull all contigs with a hit to an RdRp from fasta file to perform blastx against nr

mamba activate bbmap
filterbyname.sh in=../resources/Hugo_metaT.assembly.fa out=../results/concat_files/metaT.Rdrphits.fa \
names=all_contigs_rdrphit.txt include=t

# perform blastx using diamond

mamba activate diamond
diamond blastx -q metaT.Rdrphits.fa \
--min-orf 600 --max-target-seqs 1 \
-e 1e-5 --threads 50 \
--very-sensitive \
--db nr.dmnd \
-o ../diamond/RdRp_hits.to.nr.csv

## manually compare output hits to NCBI protein names, remove all that have hit to non-viral protein.

### Step 6: Cluster resulting proteins with match at 99% identity using cdhit for alignment

Before alignment: From contigs that only had a blastx hit, and no HMM hit, find the specific proteins that are an RdRp. 
- take out the contigs that had only a blastx hit (n=42)
- predict proteins (n=46) and we want only those 42 RdRps
- run blastp against RdRp db 
- results in 42 matches! Good

In [ ]:
# clear protein headers from pyrodigal (remove everything after space)
cut -d ' ' -f1 ../protein/Hugo.metaT_all.faa > ../protein/Hugo.metaT_all.trim.faa 

# pull out all proteins that are predicted to be RdRp (5,139)

mamba activate seqkit
seqkit grep -n -f all_proteins_rdrphit.txt Hugo.metaT_all.cdhit.99.trim.faa -o metaT.Rdrphits.faa

# deduplicate at 99% ident, 3,859 sequences left

mamba activate cdhit
cd-hit -i  metaT.Rdrphits.faa -o  metaT.Rdrphits.dedup.faa \
-c 0.99 -T 1

In [ ]:
# Step 7: Use clustalo for alignment
mamba activate clustalo
clustalo --p1 ../resources/RdRp-scan/Phylogenies/RdRp-scan.CLUSTALO_0.4.fasta \
--auto -i ./concatenated_files/metaT.Rdrphits.dedup.faa -t Protein --threads=12 \
-o ./alignments/alignrdrp.clustalo

TODO:
- trim alignments (TrimAL)
- best fit model AA substitution (ProtTest)
- phylo trees (FastTree)

In [ ]:
# srun
srun --account=ctbrowngrp -p med2 -J clustalo -t 4:00:00 -c 12 --mem=30gb --pty bash

In [ ]:
# Make sure to export each new env into a yml
conda env export > environment.yml

see https://github.com/AnneliektH/2024-caleb-snakemake/ on how to call on them